## Associate for every lap, the average weather data info

In [10]:
import fastf1 as ff1
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm  # Per la barra di progresso (opzionale)
from scipy.stats import circmean

# Select a driver

In [11]:
piloti_per_anno = {
    2015: ["HAM", "ROS", "VET", "RAI", "BOT", "MAS", "ALO", "BUT", "RIC", "KVY", "PER", "HUL", "GRO", "MAL", "SAI", "VER", "ERI", "NAS"],
    2016: ["HAM", "ROS", "VET", "RAI", "BOT", "MAS", "ALO", "BUT", "RIC", "VER", "PER", "HUL", "GRO", "GUT", "SAI", "KVY", "ERI", "NAS", "WEH", "OCO"],
    2017: ["VET", "HAM", "BOT", "RAI", "VER", "RIC", "PER", "OCO", "HUL", "SAI", "STR", "MAS", "ALO", "VAN", "GRO", "MAG", "ERI", "WEH"],
    2018: ["HAM", "VET", "RAI", "BOT", "VER", "RIC", "PER", "OCO", "HUL", "SAI", "LEC", "ERI", "GRO", "MAG", "ALO", "VAN", "STR", "SIR"],
    2019: ["HAM", "BOT", "VET", "LEC", "VER", "GAS", "ALB", "RIC", "HUL", "SAI", "PER", "STR", "RAI", "GIO", "GRO", "MAG", "KUB", "RUS"],
    2020: ["HAM", "BOT", "VER", "ALB", "LEC", "VET", "PER", "STR", "RIC", "OCO", "SAI", "NOR", "RAI", "GIO", "GRO", "MAG", "RUS", "LAT"],
    2021: ["VER", "HAM", "BOT", "PER", "LEC", "SAI", "RIC", "NOR", "GAS", "TSU", "OCO", "ALO", "VET", "STR", "RAI", "GIO", "RUS", "LAT", "MSC", "MAZ"],
    2022: ["VER", "PER", "LEC", "SAI", "HAM", "RUS", "NOR", "RIC", "OCO", "ALO", "GAS", "TSU", "VET", "STR", "BOT", "ZHO", "MSC", "MAG", "ALB", "LAT"],
    2023: ["VER", "PER", "HAM", "RUS", "LEC", "SAI", "NOR", "PIA", "ALO", "STR", "GAS", "OCO", "ALB", "SAR", "BOT", "ZHO", "MAG", "HUL"],
    2024: ["VER", "PER", "HAM", "RUS", "LEC", "SAI", "NOR", "PIA", "ALO", "STR", "GAS", "OCO", "ALB", "SAR", "BOT", "ZHO", "MAG", "HUL", "RIC", "TSU"]
}

gp_per_anno = {
    2015: ["Australia", "Malaysia", "China", "Bahrain", "Spain", "Monaco", "Canada", "Austria", "Great Britain", "Hungary", "Belgium", "Monza", "Singapore", "Japan", "Russia", "Austin", "Mexico", "Brazil", "Abu Dhabi"],
    2016: ["Australia", "Bahrain", "China", "Russia", "Spain", "Monaco", "Canada", "Azerbaijan", "Austria", "Great Britain", "Hungary", "Germany", "Belgium", "Monza", "Singapore", "Malaysia", "Japan", "Austin", "Mexico", "Brazil", "Abu Dhabi"],
    2017: ["Australia", "China", "Bahrain", "Russia", "Spain", "Monaco", "Canada", "Azerbaijan", "Austria", "Great Britain", "Hungary", "Belgium", "Monza", "Singapore", "Malaysia", "Japan", "Austin", "Mexico", "Brazil", "Abu Dhabi"],
    2018: ["Australia", "Bahrain", "China", "Azerbaijan", "Spain", "Monaco", "Canada", "France", "Austria", "Great Britain", "Germany", "Hungary", "Belgium", "Monza", "Singapore", "Russia", "Japan", "Austin", "Mexico", "Brazil", "Abu Dhabi"],
    2019: ["Australia", "Bahrain", "China", "Azerbaijan", "Spain", "Monaco", "Canada", "France", "Austria", "Great Britain", "Germany", "Hungary", "Belgium", "Monza", "Singapore", "Russia", "Japan", "Mexico", "Austin", "Brazil", "Abu Dhabi"],
    2020: ["Austria", "Styria", "Hungary", "Great Britain", "70thAnniversary", "Spain", "Belgium", "Monza", "Tuscany", "Russia", "Eifel", "Portugal", "Imola", "Turkey", "Bahrain", "Sakhir", "Abu Dhabi"],
    2021: ["Bahrain", "Imola", "Portugal", "Spain", "Monaco", "Azerbaijan", "France", "Styria", "Austria", "Great Britain", "Hungary", "Belgium", "Zandvoort", "Monza", "Russia", "Turkey", "Austin", "Mexico", "Brazil", "Qatar", "Saudi Arabia", "Abu Dhabi"],
    2022: ["Bahrain", "Saudi Arabia", "Australia", "Imola", "Miami", "Spain", "Monaco", "Azerbaijan", "Canada", "Great Britain", "Austria", "France", "Hungary", "Belgium", "Zandvoort", "Monza", "Singapore", "Japan", "Austin", "Mexico", "Brazil", "Abu Dhabi"],
    2023: ["Bahrain", "Saudi Arabia", "Australia", "Azerbaijan", "Miami", "Monaco", "Spain", "Canada", "Austria", "Great Britain", "Hungary", "Belgium", "Zandvoort", "Monza", "Singapore", "Japan", "Qatar", "Austin", "Mexico", "Brazil", "LasVegas", "Abu Dhabi"],
    2024: ["Bahrain", "Saudi Arabia", "Australia", "Japan", "China", "Miami", "Imola", "Monaco", "Canada", "Spain", "Austria", "Great Britain", "Hungary", "Belgium", "Zandvoort", "Monza", "Azerbaijan", "Singapore", "Austin", "Mexico", "Brazil", "LasVegas", "Qatar", "Abu Dhabi"]
}
# Funzione per scegliere da un menu
def scegli_opzione(opzioni, messaggio):
    print(f"\n{messaggio}:")
    for i, opzione in enumerate(opzioni, 1):
        print(f"{i}. {opzione}")
    
    while True:
        try:
            scelta = int(input("Inserisci il numero: "))
            if 1 <= scelta <= len(opzioni):
                return opzioni[scelta - 1]
            print("Numero non valido.")
        except ValueError:
            print("Input non valido. Riprova.")

# --- Scelta anno ---
year = scegli_opzione(list(gp_per_anno.keys()), "Scegli l'anno")

# --- Scelta GP ---
gp = scegli_opzione(gp_per_anno[anno], f"Scegli il GP del {anno}")

# --- Scelta pilota (lista fissa per l'anno) ---
driver = scegli_opzione(piloti_per_anno[anno], f"Scegli il pilota per il {anno}")

# Risultato finale
print(f"\nHai selezionato: {driver} al GP di {gp} nel {year}!")


Scegli l'anno:
1. 2015
2. 2016
3. 2017
4. 2018
5. 2019
6. 2020
7. 2021
8. 2022
9. 2023
10. 2024
Input non valido. Riprova.
Input non valido. Riprova.
Input non valido. Riprova.
Input non valido. Riprova.
Input non valido. Riprova.
Input non valido. Riprova.
Input non valido. Riprova.


NameError: name 'anno' is not defined

# Create a copy of driver dataframe

In [ ]:
session = ff1.get_session(2023, 'Monaco', 'R')
sessione = session.load()
weather_data = session.weather_data
driver = 'SAR'
driver_laps = session.laps.pick_drivers(driver)

req         WARNING 	DEFAULT CACHE ENABLED! (24.0 KB) C:\Users\calin\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Monaco Grand Prix - Race [v3.4.5]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track 

# Add data from weather df

In [ ]:
driver_laps['AirTemp'] = np.nan
driver_laps['Humidity'] = np.nan
driver_laps['Pressure'] = np.nan
driver_laps['Rainfall'] = np.nan
driver_laps['Track_Temp'] = np.nan
driver_laps['WindDirection'] = np.nan
driver_laps['WindSpeed'] = np.nan
driver_laps['Cumulative_Tire_Distance'] = np.nan
driver_laps['Driver_Meters'] = np.nan

# Get the circuit length in meters
location = session.event.Location
print(location)
circuit_length = circuits_length.get(location, 0)

driver_meters = 0

#Creo il contatore partendo dalla prima riga
for i in range(len(driver_laps)):
    lap_number = driver_laps.iloc[i]['LapNumber']
    tire_life = driver_laps.iloc[i]['TyreLife']

    #Definisco la variabile lap_start_driver, quando inizia il giro in corso(i)
    lap_start_driver = driver_laps['LapStartTime'].iloc[i]
    #Definisco la variabile lap_finish_driver, quando finisce il giro in corso(i)
    lap_finish_driver = driver_laps['Time'].iloc[i]
    #Stampo i valori
    print('Il giro inizia : ', lap_start_driver)
    print('Il giro finisce a: ', lap_finish_driver)

    print('Il numero del giro è: ', lap_number)
    print('La vita della gomma è: ', tire_life)

    # Seleziona solo le righe con Time tra lap_start_driver e lap_finish_driver
    weather_first_lap = weather_data[
        (weather_data['Time'] >= lap_start_driver) & 
        (weather_data['Time'] <= lap_finish_driver)
    ]
    print(weather_first_lap)

    ###############################################################################

    temperatura = 0
    humidity = 0
    rain = 0
    track_temp = 0
    pressure = 0
    wind_speed = 0
    wind_direction = 0
    for j in range(len(weather_first_lap)):

        #Calcolo la temperatura media
        temperatura = temperatura + weather_first_lap['AirTemp'].iloc[j]
        
        #Calcolo umidità media
        humidity = humidity + weather_first_lap['Humidity'].iloc[j]

        #Calocolo pioggia True/False
        if weather_first_lap['Rainfall'].iloc[j] == True:
            rain = 1
        
        #Calculate AVG track Temp
        track_temp = track_temp + weather_first_lap['TrackTemp'].iloc[j]

        #Calculate AVG Pressure
        pressure = pressure + weather_first_lap['Pressure'].iloc[j]

        #Calculate AVG Wind Speed
        wind_speed = wind_speed + weather_first_lap['WindSpeed'].iloc[j]

        #Calculate AVG Wind Direction
        wind_direction = wind_direction + weather_first_lap['WindDirection'].iloc[j]


        j = j + 1

    
    air_temp_avg = temperatura / len(weather_first_lap)
    air_temp_01 = ( air_temp_avg * 1 ) / 61.0  # 61 is the highst value of air temperature in Malaysia relevant to the 2015 season
    print('Temperatura_media ',air_temp_avg)
    humidity_avg = humidity / len(weather_first_lap)
    humidity_01 = ( humidity_avg * 1 ) / 90.0  # 90 is the highst value of humidity in Singapore relevant to the 2017 season
    print('Umidità ',humidity_avg)
    print('Piove? ',rain)
    trackTemp_AVG = track_temp / len(weather_first_lap)
    trackTemp_01 = ( trackTemp_AVG * 1 ) / 61.0  # 61 is the highst value of track temperature in Malaysia relevant to the 2016 season
    print('TrackTemp AVG ', trackTemp_AVG)
    pressure_avg = pressure / len(weather_first_lap)
    pressure_01 = ( pressure_avg * 1 ) / 1013.0  # 1013 is the highst value of pressure in Singapore relevant to the 2017 season
    print('Pressure AVG', pressure_avg)
    wind_speed_avg = wind_speed / len(weather_first_lap)
    wind_speed_01 = ( wind_speed_avg * 1 ) / 80.0  # 10 is the highst value of wind speed in Singapore relevant to the 2017 season
    print('Wind Speed AVG', wind_speed_avg)
    wind_direction_avg = wind_direction / len(weather_first_lap)
    wind_direction_01 = ( wind_direction_avg * 1 ) / 360.0  # 360 is the highst value of wind direction 
    print('Wind Direction AVG', wind_direction_avg)

    if(tire_life and tire_life > 0 and not tire_life == np.nan):

        cumulative_tire_distance = int(tire_life * circuit_length)
        driver_meters = int(lap_number * circuit_length)
        print(f'Cumulative Tire Distance = {cumulative_tire_distance}: {tire_life} * {circuit_length}')
        print(f'Driver Meters = {driver_meters}: {lap_number} * {circuit_length}')

    else:
        cumulative_tire_distance = 0
        driver_meters = 0

    cumulative_tire_distance_01 = ( cumulative_tire_distance * 1 ) / 30466.0  # 30466 is the highst value of cumulative tire distance in Singapore relevant to the 2017 season
    driver_meters_01 = ( driver_meters * 1 ) / 470884.0  # 470884 is the highst value of driver meters in Singapore relevant to the 2017 season

    driver_laps.iloc[i, driver_laps.columns.get_loc('AirTemp')] = air_temp_01
    driver_laps.iloc[i, driver_laps.columns.get_loc('Humidity')] = humidity_01
    driver_laps.iloc[i, driver_laps.columns.get_loc('Pressure')] = pressure_01
    driver_laps.iloc[i, driver_laps.columns.get_loc('Rainfall')] = rain
    driver_laps.iloc[i, driver_laps.columns.get_loc('Track_Temp')] = trackTemp_01
    driver_laps.iloc[i, driver_laps.columns.get_loc('WindDirection')]  = wind_direction_01
    driver_laps.iloc[i, driver_laps.columns.get_loc('WindSpeed')] = wind_speed_01
    driver_laps.iloc[i, driver_laps.columns.get_loc('Cumulative_Tire_Distance')] = cumulative_tire_distance_01
    driver_laps.iloc[i, driver_laps.columns.get_loc('Driver_Meters')] = driver_meters_01


C:\Users\calin\AppData\Local\Temp\ipykernel_13952\1385738569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_laps['AirTemp'] = np.nan
C:\Users\calin\AppData\Local\Temp\ipykernel_13952\1385738569.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_laps['Humidity'] = np.nan
C:\Users\calin\AppData\Local\Temp\ipykernel_13952\1385738569.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

Il giro inizia :  0 days 01:02:02.950000
Il giro finisce a:  0 days 01:03:40.909000
                     Time  AirTemp  Humidity  Pressure  Rainfall  TrackTemp  \
62 0 days 01:02:42.715000     25.2      44.0    1013.4     False       43.1   

    WindDirection  WindSpeed  
62            153        0.8  
Temperatura_media  25.2
Umidità  44.0
Piove?  0
TrackTemp AVG  43.1
Pressure AVG 1013.4
Wind Speed AVG 0.8
Wind Direction AVG 153.0
Il giro inizia :  0 days 01:03:40.909000
Il giro finisce a:  0 days 01:05:00.567000
                     Time  AirTemp  Humidity  Pressure  Rainfall  TrackTemp  \
63 0 days 01:03:42.714000     25.2      44.0    1013.4     False       43.1   
64 0 days 01:04:42.729000     25.2      45.0    1013.5     False       42.8   

    WindDirection  WindSpeed  
63             48        1.0  
64             66        0.8  
Temperatura_media  25.2
Umidità  44.5
Piove?  0
TrackTemp AVG  42.95
Pressure AVG 1013.45
Wind Speed AVG 0.9
Wind Direction AVG 57.0
Il giro inizia 

Show df copy

In [16]:
driver_laps

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,DeletedReason,FastF1Generated,IsAccurate,AirTemp,Humidity,Pressure,Rainfall,Track_Temp,WindDirection,WindSpeed
442,0 days 01:03:40.909000,SAR,2,0 days 00:01:37.689000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:44.183000,...,,False,False,0.413115,0.488889,1.000395,0.0,0.706557,0.425000,0.010000
443,0 days 01:05:00.567000,SAR,2,0 days 00:01:19.658000,2.0,1.0,NaT,NaT,0 days 00:00:21.209000,0 days 00:00:37.307000,...,,False,True,0.413115,0.494444,1.000444,0.0,0.704098,0.158333,0.011250
444,0 days 01:06:20.786000,SAR,2,0 days 00:01:20.219000,3.0,1.0,NaT,NaT,0 days 00:00:21.279000,0 days 00:00:37.588000,...,,False,True,0.414754,0.500000,1.000395,0.0,0.701639,0.050000,0.012500
445,0 days 01:07:41.032000,SAR,2,0 days 00:01:20.246000,4.0,1.0,NaT,NaT,0 days 00:00:21.243000,0 days 00:00:37.688000,...,,False,True,0.416393,0.500000,1.000494,0.0,0.701639,0.319444,0.012500
446,0 days 01:09:01.246000,SAR,2,0 days 00:01:20.214000,5.0,1.0,NaT,NaT,0 days 00:00:21.315000,0 days 00:00:37.650000,...,,False,True,0.417213,0.488889,1.000494,0.0,0.695082,0.406944,0.014375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,0 days 02:46:16.048000,SAR,2,0 days 00:01:32.354000,72.0,4.0,NaT,NaT,0 days 00:00:26.671000,0 days 00:00:44.166000,...,,False,True,0.390164,0.633333,1.000000,0.0,0.491803,0.347222,0.013750
514,0 days 02:47:43.185000,SAR,2,0 days 00:01:27.137000,73.0,4.0,NaT,NaT,0 days 00:00:22.512000,0 days 00:00:42.573000,...,,False,True,0.390984,0.627778,1.000000,0.0,0.497541,0.394444,0.018750
515,0 days 02:49:11.139000,SAR,2,0 days 00:01:27.954000,74.0,4.0,NaT,NaT,0 days 00:00:23.755000,0 days 00:00:42.097000,...,,False,True,0.391803,0.633333,0.999901,0.0,0.503279,0.377778,0.021250
516,0 days 02:50:39.107000,SAR,2,0 days 00:01:27.968000,75.0,4.0,NaT,NaT,0 days 00:00:22.797000,0 days 00:00:41.457000,...,,False,True,0.393443,0.655556,0.999901,0.0,0.503279,0.400000,0.016250


Generate csv from df

In [17]:
driver_laps.to_csv(f'driver_laps_{driver}2.csv', index=False)